In [2]:
import mlflow

import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression , Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import pickle

Run following command in terminal


mlflow server --backend-store-uri sqlite:///backend.db

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000") 
mlflow.set_experiment("Electricity Demand Prediction")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1723988056227, experiment_id='1', last_update_time=1723988056227, lifecycle_stage='active', name='Electricity Demand Prediction', tags={}>

In [4]:
def read_dataframe(filename):
    df = pd.read_csv(filename)
    df['date'] = pd.to_datetime(df['date'])
    df['demand'] = pd.to_numeric(df['demand'], errors='coerce').astype('float')
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hr'] = df['date'].dt.hour
    df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Sunday=6
    df['is_weekend'] = df['date'].dt.dayofweek >= 5  # True if weekend, False otherwise
    
    holidays = calendar().holidays(start=df['date'].min(), end=df['date'].max())
    df['holiday'] = df['date'].isin(holidays).astype(int)
    #display(df.head())
    #print(df.dtypes)
    
    
    return df

In [5]:
dataset_path = "dataset.csv"

In [6]:
df = read_dataframe(dataset_path)

In [7]:
# Define features and target
X = df.drop('demand', axis=1)
X = X.drop('date', axis=1)

y = df['demand']

In [8]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [10]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        mlflow.xgboost.autolog()
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [12]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)
print(best_result)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2024/08/18 13:35:12 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.

/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:35:12] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:2111.62500                        
[1]	validation-rmse:1481.76305                        
[2]	validation-rmse:1121.70328                        
[3]	validation-rmse:927.94123                         
[4]	validation-rmse:831.86934                         
[5]	validation-rmse:785.41344                         
[6]	validation-rmse:764.88656                         
[7]	validation-rmse:755.85112                         
[8]	validation-rmse:752.04343                         
[9]	validation-rmse:750.15170                         
[10]	validation-rmse:748.09418                        
[11]	validation-rmse:747.22806                        
[12]	validation-rmse:746.36787                        
[13]	validation-rmse:745.78796                        
[14]	validation-rmse:745.40980                        
[15]	validation-rmse:745.02851                        
[16]	validation-rmse:744.72228                        
[17]	validation-rmse:744.52083                        
[18]	valid

2024/08/18 13:35:25 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/08/18 13:35:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:35:25] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/08/18 13:35:29 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared

  2%|▏         | 1/50 [00:17<14:30, 17.76s/trial, best loss: 743.6976692967022]

...

2024/08/18 13:51:35 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.



/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:51:35] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:2938.11751
[1]	validation-rmse:2742.19647                                                   
[2]	validation-rmse:2562.43889                                                   
[3]	validation-rmse:2396.50813                                                   
[4]	validation-rmse:2243.59942                                                   
[5]	validation-rmse:2102.89174                                                   
[6]	validation-rmse:1972.98856                                                   
[7]	validation-rmse:1853.84068                                                   
[8]	validation-rmse:1744.84769                                                   
[9]	validation-rmse:1645.29560                                                   
[10]	validation-rmse:1553.76522                                                  
[11]	validation-rmse:1470.02815                                                  
[12]	validation-rmse:1393.82982                                    

2024/08/18 13:52:33 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/08/18 13:52:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:52:33] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."



: 

In [11]:
new_observation = pd.DataFrame({
    'temperature' : [21],
    'year': [2024],
    'month': [8],
    'day': [12],
    'hr': [3],
    'day_of_week': [1],
    'is_weekend': [0],
    'holiday': [0]
})

In [12]:
import mlflow
logged_model = 'runs:/0ac14477b69842e195919d1cab69d066/model'

# Load model as a PyFuncModel
loaded_model = mlflow.pyfunc.load_model(logged_model)



/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


array([20198.39], dtype=float32)

In [13]:
loaded_model.predict(pd.DataFrame(new_observation))

array([20198.39], dtype=float32)